🚀 Dataset Toolkit - Gelbooru Scraper & Data Prep in Colab🤖✨


**Key Features:**

*   **🌐 Gelbooru Scraper:**  Download images and associated tags directly from Gelbooru based on your search queries, with extensive filtering options.
*   **🏷️ Tag Management & Cleaning:** Tools to clean and standardize your dataset tags, including:
    *   Removing duplicate tags
    *   Cleaning tag structure (spaces, parentheses)
    *   Replacing specific tags
    *   Removing unwanted tags
    *   Auto-tagging images using WD Tagger v3
*   **👯 Duplicate Image Finder:**  Identify and remove visually similar or semantically duplicate images to ensure dataset quality.
*   **🔍 Tag Analyzer:**  Gain insights into your dataset's tags with frequency analysis, category breakdowns, and powerful search capabilities, including wildcard search.
*   **🧹 Dataset Cleaner:**  Remove orphaned tag files, video files, non-image files, and images with extreme aspect ratios or resolutions.
*   **🗂️ File Management:**  Easily move or copy your processed dataset within Google Drive and count dataset statistics.



| |Github|
|:--|:-:|
| 🏠 **Profile** | [![GitHub](https://img.shields.io/badge/GitHub-%23121011.svg?logo=github&logoColor=white)](https://github.com/xLegende)|
| 📘 **Repos** | [![GitHub](https://img.shields.io/badge/GitHub-%23121011.svg?logo=github&logoColor=white)](https://github.com/xLegende/dataset_colab)|

In [ ]:
#@title ⭕ Mount Drive and Define Directories

import os
from google.colab import drive

# --- ANSI Color Codes ---
COLOR_RESET = "\033[0m"
COLOR_SUCCESS = "\033[32m"  # Green for success
COLOR_INFO = "\033[36m"     # Cyan for info messages

# --- Emojis ---
EMOJI_DRIVE = " ☁️" # Cloud for Drive
EMOJI_FOLDER = " 🗂️" # Folder for Directories
EMOJI_CHECKMARK = " ✅" # Checkmark for Success

# Mount Google Drive
print(f"{EMOJI_DRIVE}{COLOR_INFO} Mounting Google Drive...{COLOR_RESET}")
drive.mount('/content/drive')
print(f"{EMOJI_CHECKMARK}{COLOR_SUCCESS} Google Drive mounted successfully!{COLOR_RESET}")

# Define base output directory in Google Drive - you can change this here or in the form below
OUTPUT_BASE_DIR = "/content/drive/MyDrive/" #@param {type:"string"}
if not OUTPUT_BASE_DIR:
    OUTPUT_BASE_DIR = "/content/drive/MyDrive/"

# Define subdirectory for images - you can change this here or in the form below
subdirectory = "images" #@param {type:"string"}

# Construct full directory path in Google Drive
full_directory = os.path.join(OUTPUT_BASE_DIR, subdirectory)

# Ensure base directory and subdirectory exist in Google Drive
os.makedirs(OUTPUT_BASE_DIR, exist_ok=True)
os.makedirs(full_directory, exist_ok=True)

print(f"\n--- {EMOJI_FOLDER} Directories Initialized {EMOJI_FOLDER} ---")
print(f"{EMOJI_FOLDER} Base output directory set to: {COLOR_INFO}{OUTPUT_BASE_DIR}{COLOR_RESET}")
print(f"{EMOJI_FOLDER} Subdirectory for images: {COLOR_INFO}{subdirectory}{COLOR_RESET}")
print(f"{EMOJI_FOLDER} Full output directory: {COLOR_INFO}{full_directory}{COLOR_RESET}")
print(f"{EMOJI_CHECKMARK}{COLOR_SUCCESS} Directories initialized successfully!{COLOR_RESET}")

In [ ]:
#@title 🛃 Gelbooru Scraper

import requests
import json
import time
# No need to import drive or os here, already done in the init cell

# Colab parameters with default values
search_query = "-lowres *res -video -mp4 -monochrome -greyscale score:>5 -*english* -speech_bubble -comic -*text* " #@param {type:"string"}
limit = 5 #@param {type:"integer"}
# subdirectory = "images" # Removed, defined in init cell
include_general_tags = False #@param {type:"boolean"}
include_artist_tags = True #@param {type:"boolean"}
include_metadata_tags = True #@param {type:"boolean"}
include_character_tags = True #@param {type:"boolean"}
include_copyright_tags = False #@param {type:"boolean"}
filter_unknown_tags = True #@param {type:"boolean"}
trigger_word = "" #@param {type:"string"}
exclude_tags_str = "tagme, commentary_request, commentary, translation_request, official_art, english_commentary, commission, artist_request, non-web_source, skeb_commission, paid_reward_available, second-party_source, hashtag-only_commentary, bad_id, bad_twitter_id, character_request, translated, partial_commentary, copyright_request, chinese_commentary, korean_commentary, mixed-language_commentary, third-party_edit, revision, photoshop_(medium), source_request, symbol-only_commentary, variant_set, textless_version, third-party_source, bad_pixiv_id, commissioner_upload, making-of_available, clothing_request, self-upload, slideshow_animation, novel_illustration,photo-referenced, duplicate, huge_filesize, corrupted_twitter_file,flower_request, pixiv_commission, check_copyright, bad_link, food_request, vgen_commission, check_character, large_variant_set, ai-assisted, bad_fanbox_id, clip_studio_paint_(medium), paid_reward, check_artist, bad_source, fantia_reward, nude_filter, protected_link, inactive_account, derivative_work, artist_collaboration, image_sample, webp-to-png_conversion, weapon_request, psd_available, check_commentary, md5_mismatch, spoilers, archived_source, alt_text, merchandise_available, annotated, pixiv_sample, hair_ornament_request, live2d," #@param {type:"string"}

exclude_tags = [tag.strip() for tag in exclude_tags_str.split(',') if tag.strip()]

def scrape_gelbooru(query, limit=100, directory=full_directory, include_general=True, include_artists=False, include_metadata=False, include_characters=False, include_copyright=False, filter_unknown=False, trigger_word="", exclude_tags=[]): # Use full_directory directly
    """
    Scrapes images from Gelbooru, providing options for a trigger word, tag exclusion,
    artist tags, metadata tags, character tags, copyright tags, and filtering unknown tags.
    Saves images and tags to Google Drive.

    Args:
        query (str): The search query.
        limit (int): The maximum number of images to download.
        directory (str): The directory to save images and tag files (relative to Google Drive base).
        include_general (bool): Whether to include general tags.
        include_artists (bool): Whether to include artist tags.
        include_metadata (bool): Whether to include metadata tags.
        include_characters (bool): Whether to include character tags.
        include_copyright (bool): Whether to include copyright tags.
        filter_unknown (bool): Whether to filter out tags of unknown type.
        trigger_word (str): A trigger word to be added as the first tag.
        exclude_tags (list): A list of tags to exclude from the tag file.
    """

    api_url = "https://gelbooru.com/index.php"
    page = 0
    downloaded_count = 0

    # Directory is already created in the init cell
    print(f"Saving images and tags to: {directory}") # directory is now full_directory

    while downloaded_count < limit:
        params = {
            "page": "dapi",
            "s": "post",
            "q": "index",
            "tags": query,
            "json": 1,
            "pid": page,
            "limit": 100,
        }

        try:
            response = requests.get(api_url, params=params)
            response.raise_for_status()
            data = response.json()

            if not data or 'post' not in data or not data['post']:
                print("No more images found.")
                break

            for post in data['post']:
                if downloaded_count >= limit:
                    break

                image_url = post['file_url']
                image_id = post['id']
                base_filename = f"{image_id}"
                image_filename = f"{base_filename}{os.path.splitext(image_url)[1]}"
                image_path = os.path.join(directory, image_filename) # Use directory (full_directory)
                tag_filename = f"{base_filename}.txt"
                tag_path = os.path.join(directory, tag_filename) # Use directory (full_directory)

                # Get tag information with categories using the API
                tag_data = get_tag_data(post['tags'])
                tags_to_write = []

                # Add trigger word if provided
                if trigger_word:
                    tags_to_write.append(trigger_word)

                for tag_info in tag_data:
                    tag_name = tag_info['name']
                    tag_type = tag_info['category']

                    # Exclude tags based on the exclude_tags list
                    if tag_name in exclude_tags:
                        continue

                    # Filter tags based on type and boolean flags
                    if (tag_type == "general" and not include_general) or \
                       (tag_type == "artist" and not include_artists) or \
                       (tag_type == "meta" and not include_metadata) or \
                       (tag_type == "character" and not include_characters) or \
                       (tag_type == "copyright" and not include_copyright) or \
                       (filter_unknown and tag_type == "unknown"):  # Filter unknown tags
                        continue
                    else:
                        tags_to_write.append(tag_name)

                tags_string = ', '.join(tags_to_write)

                try:
                    image_response = requests.get(image_url, stream=True)
                    image_response.raise_for_status()

                    with open(image_path, 'wb') as image_file:
                        for chunk in image_response.iter_content(chunk_size=8192):
                            image_file.write(chunk)

                    with open(tag_path, 'w') as tag_file:
                        tag_file.write(tags_string)

                    print(f"Downloaded: {image_filename} with tags in {tag_filename} to '{subdirectory}'")
                    downloaded_count += 1

                except requests.exceptions.RequestException as e:
                    print(f"Error downloading {image_url}: {e}")

                time.sleep(0.1)

            page += 1

        except requests.exceptions.RequestException as e:
            print(f"Error fetching data from API: {e}")
            break
        except json.JSONDecodeError as e:
            print(f"Error decoding JSON: {e}")
            break

def get_tag_data(tags_string):
    """
    Retrieves tag data with categories using the Gelbooru API.

    Args:
        tags_string (str): A space-separated string of tags.

    Returns:
        list: A list of dictionaries, where each dictionary represents a tag
              and contains 'name' and 'category' keys.
    """

    api_url = "https://gelbooru.com/index.php"
    params = {
        "page": "dapi",
        "s": "tag",
        "q": "index",
        "json": 1,
        "names": tags_string
    }

    try:
        response = requests.get(api_url, params=params)
        response.raise_for_status()
        tag_data = response.json()

        # Check if the response is a dictionary and convert it to a list
        if isinstance(tag_data, dict) and 'tag' in tag_data:
            tag_data = tag_data['tag']
        elif not isinstance(tag_data, list):
            print(f"Unexpected API response format: {tag_data}")
            return []

        # Map Gelbooru's numerical tag types to category names
        tag_type_mapping = {
            0: "general",
            1: "artist",
            3: "copyright",
            4: "character",
            5: "meta",
        }

        # Extract tag names and categories
        tag_info = []
        for tag in tag_data:
            tag_info.append({
                "name": tag["name"],
                "category": tag_type_mapping.get(tag["type"], "unknown")
            })

        return tag_info

    except requests.exceptions.RequestException as e:
        print(f"Error fetching tag data from API: {e}")
        return []
    except json.JSONDecodeError as e:
        print(f"Error decoding JSON for tag data: {e}")
        return []

# Main execution using Colab parameters
if __name__ == "__main__": # This block will only run if you run as a script, not in colab directly, so removing it
    scrape_gelbooru(
        search_query,
        limit=limit,
        # directory=subdirectory, # Removed, using full_directory directly in function definition
        include_general=include_general_tags,
        include_artists=include_artist_tags,
        include_metadata=include_metadata_tags,
        include_characters=include_character_tags,
        include_copyright=include_copyright_tags,
        filter_unknown=filter_unknown_tags,
        trigger_word=trigger_word,
        exclude_tags=exclude_tags
    )

In [ ]:
#@title 🤖 Auto Tagger (WD Tagger v3)

import os
import timm
from PIL import Image
import numpy as np
import torch
import pandas as pd # Import pandas
from huggingface_hub import hf_hub_download # Import hf_hub_download
from timm.data import create_transform, resolve_data_config # Import timm transforms
from torch.nn import functional as F # Import F for sigmoid

# No need to import drive or define directories again
# Define directory using the initialized full_directory
# full_directory is assumed to be defined in the "Initialization Cell"

# Colab Parameters
enable_auto_tagging = True  #@param {type:"boolean"}
general_tag_threshold = 0.4  #@param {type:"slider", min:0.0, max:1.0, step:0.05}
character_tag_threshold = 0.8  #@param {type:"slider", min:0.0, max:1.0, step:0.05}
tag_saving_option = "Overwrite"  #@param ["Overwrite", "Append"]

MODEL_REPO_MAP = { # Model repo map from example script
    "vit": "SmilingWolf/wd-vit-tagger-v3",
    "swinv2": "SmilingWolf/wd-swinv2-tagger-v3",
    "convnext": "SmilingWolf/wd-convnext-tagger-v3",
}
MODEL_NAME = "swinv2" #@param ["vit", "swinv2", "convnext"]

def load_wd_tagger(model_name_key=MODEL_NAME): # Modified to take model_name_key
    """Loads WD Tagger model using timm library."""
    try:
        repo_id = MODEL_REPO_MAP.get(model_name_key) # Get repo_id from MODEL_REPO_MAP
        model = timm.create_model("hf-hub:" + repo_id).eval() # Load model from timm with hf-hub prefix
        state_dict = timm.models.load_state_dict_from_hf(repo_id) # Load state_dict
        model.load_state_dict(state_dict) # Load state_dict into model

        print(f"✅ WD Tagger ({model_name_key}) model loaded successfully.")
        return model
    except Exception as e:
        print(f"❌ Error loading WD Tagger ({model_name_key}) model: {e}")
        return None

def pil_ensure_rgb(image: Image.Image) -> Image.Image:
    # convert to RGB/RGBA if not already (deals with palette images etc.)
    if image.mode not in ["RGB", "RGBA"]:
        image = image.convert("RGBA") if "transparency" in image.info else image.convert("RGB")
    # convert RGBA to RGB with white background
    if image.mode == "RGBA":
        canvas = Image.new("RGBA", image.size, (255, 255, 255))
        canvas.alpha_composite(image)
        image = canvas.convert("RGB")
    return image

def pil_pad_square(image: Image.Image) -> Image.Image:
    w, h = image.size
    # get the largest dimension so we can pad to a square
    px = max(image.size)
    # pad to square with white background
    canvas = Image.new("RGB", (px, px), (255, 255, 255))
    canvas.paste(image, ((px - w) // 2, (px - h) // 2))
    return canvas


def load_labels_hf(repo_id): # Modified to take repo_id
    """Loads labels from HuggingFace Hub."""

    labels_path_hf = hf_hub_download(repo_id=repo_id, filename="selected_tags.csv",  local_dir=".") # Download labels CSV

    labels_df = pd.read_csv(labels_path_hf, usecols=["name", "category"]) # Load labels using pandas
    label_names = labels_df["name"].tolist()
    categories = labels_df["category"].tolist()

    labels = {}
    labels['names'] = label_names
    labels['rating'] = list(np.where(np.array(categories) == 9)[0])
    labels['general'] = list(np.where(np.array(categories) == 0)[0])
    labels['character'] = list(np.where(np.array(categories) == 4)[0])

    return labels



def get_tags(
    probs,
    labels,
    gen_threshold: float,
    char_threshold: float,
):
    # Convert indices+probs to labels
    probs_list = list(zip(labels['names'], probs)) # Use labels dict

    # First 4 labels are actually ratings
    rating_labels = dict([probs_list[i] for i in labels['rating']]) # Use labels dict

    # General labels, pick any where prediction confidence > threshold
    gen_labels = [probs_list[i] for i in labels['general']] # Use labels dict
    gen_labels = dict([x for x in gen_labels if x[1] > gen_threshold])
    gen_labels = dict(sorted(gen_labels.items(), key=lambda item: item[1], reverse=True))

    # Character labels, pick any where prediction confidence > threshold
    char_labels = [probs_list[i] for i in labels['character']] # Use labels dict
    char_labels = dict([x for x in char_labels if x[1] > char_threshold])
    char_labels = dict(sorted(char_labels.items(), key=lambda item: item[1], reverse=True))

    # Combine general and character labels, sort by confidence
    combined_names = [x for x in gen_labels]
    combined_names.extend([x for x in char_labels])

    # Convert to a string suitable for use as a training caption
    caption = ", ".join(combined_names)
    taglist = caption.replace("_", " ").replace("(", "\(").replace(")", "\)")

    return caption, taglist, rating_labels, char_labels, gen_labels


def auto_tag_image(image_path, model, labels, general_threshold, character_threshold, device="cuda" if torch.cuda.is_available() else "cpu"): # Modified to accept labels
    """Generates tags for an image using WD Tagger (timm)."""
    try:
        image = Image.open(image_path).convert("RGB")
        img_tensor = timm.data.transforms_factory.transforms_imagenet_eval(448)(image).unsqueeze(0).to(device) # CHANGED: Use 448 instead of 256
        img_tensor = img_tensor[:, [2, 1, 0]] # RGB to BGR
        model.to(device)

        with torch.no_grad():
          output = model(img_tensor)
          probs = F.sigmoid(output.cpu()).detach().numpy()[0] # Apply sigmoid and move to CPU

        character_tags, general_tags = get_tags(probs=probs, labels=labels, gen_threshold=general_threshold, char_threshold=character_threshold)[3:] # Use get_tags function

        character_tags = list(character_tags.keys()) # Extract tag names from dict
        general_tags = list(general_tags.keys()) # Extract tag names from dict


        return character_tags, general_tags

    except Exception as e:
        print(f"❌ Error tagging image {image_path}: {e}")
        return [], []

def save_tags(tag_path, character_tags, general_tags, tag_saving_option="Overwrite"):
    """Saves generated tags to a .txt file."""
    try:
        tags_string = ', '.join(character_tags + general_tags)

        if tag_saving_option == "Overwrite":
          with open(tag_path, 'w') as f:
                f.write(tags_string)
        elif tag_saving_option == "Append":
          with open(tag_path, 'a') as f:
            if os.path.getsize(tag_path) > 0: # Check if tag file is empty
                f.write(', ')
            f.write(tags_string)
        else:
            print(f"⚠️ Warning: Invalid tag saving option selected: '{tag_saving_option}'. Tags not saved for {tag_path}")

        return True
    except Exception as e:
        print(f"❌ Error saving tags to {tag_path}: {e}")
        return False

if enable_auto_tagging:
  print("🤖 Starting auto-tagging process...")

  # Install pandas if not already installed
  try:
      import pandas as pd
  except ImportError:
      print("Installing pandas...")
      !pip install pandas -q
      import pandas as pd

  # Install timm if not already installed
  try:
      import timm
  except ImportError:
      print("Installing timm...")
      !pip install timm -q
      import timm


  model = load_wd_tagger() # Load model using timm
  labels = load_labels_hf(repo_id=MODEL_REPO_MAP.get(MODEL_NAME)) # Load labels


  if model and labels: # Check if both model and labels loaded successfully
    image_extensions = ['.jpg', '.jpeg', '.png', '.webp'] # Add GIF here if needed
    tagged_count = 0
    tag_errors = 0


    print(f"✅ Labels file 'selected_tags.csv' loaded.")

    for filename in os.listdir(full_directory):
      is_image = False
      for ext in image_extensions:
          if filename.lower().endswith(ext):
              is_image = True
              break
      if is_image:
        image_path = os.path.join(full_directory, filename)
        tag_filename = os.path.splitext(filename)[0] + ".txt"
        tag_path = os.path.join(full_directory, tag_filename)

        character_tags, general_tags = auto_tag_image(image_path, model, labels, general_tag_threshold, character_tag_threshold) # Pass labels to auto_tag_image

        if save_tags(tag_path, character_tags, general_tags, tag_saving_option):
          tagged_count += 1
          print(f"✅ Tagged: {filename} -> tags saved to {tag_filename}")
        else:
          tag_errors += 1
          print(f"❌ Error: Failed to save tags for {filename}")


        print("\n--- Auto Tagging Summary ---")
        print(f"Images tagged: {tagged_count}")
        print(f"Tag errors: {tag_errors}")
        print("✅ Auto-tagging process completed .")
  else:
      print("❌ Auto-tagging process cancelled due to model loading error (timm) or labels loading error.") # Modified error message

else:
  print("Auto-tagging process is disabled. Please enable the checkbox to run this feature.")

In [ ]:
#@title 👯 Find Semantic Duplicates (using FiftyOne & CLIP)

import os
from google.colab import output
from IPython.display import clear_output

# No need to import drive or define directories again
# Define directory using the initialized full_directory
# full_directory is assumed to be defined in the "Initialization Cell"

# Feature Checkboxes
enable_semantic_duplicate_finding = True #@param {type:"boolean"}
# Cosine similarity threshold
similarity_threshold = 0.96 #@param {type:"slider", min:0.90, max:0.999, step:0.001}

images_folder = full_directory # Use the full_directory defined in initialization
root_dir = OUTPUT_BASE_DIR # Use the OUTPUT_BASE_DIR defined in initialization
project_subfolder = "cleaned_dataset" # Subfolder for exported cleaned dataset

step3_installed_flag = False # Flag to track installation

if enable_semantic_duplicate_finding:

    os.chdir(root_dir)
    model_name = "clip-vit-base32-torch"
    supported_types = (".png", ".jpg", ".jpeg")
    img_count = len(os.listdir(images_folder))
    batch_size = min(250, img_count)

    if not step3_installed_flag:
      print("🏭 Installing dependencies for semantic duplicate finding...\n")
      !pip -q install fiftyone ftfy
      !pip -q install fiftyone-db-ubuntu2204
      if not get_ipython().__dict__['user_ns']['_exit_code']:
        clear_output()
        step3_installed_flag = True
        print("✅ Dependencies installed.")
      else:
        print("❌ Error installing dependencies, attempting to continue anyway...")

    import numpy as np
    import fiftyone as fo
    import fiftyone.zoo as foz
    from fiftyone import ViewField as F
    from sklearn.metrics.pairwise import cosine_similarity

    non_images = [f for f in os.listdir(images_folder) if not f.lower().endswith(supported_types) and not f.lower().endswith(".txt")] # IGNORE .txt files
    if non_images:
      print(f"💥 Error: Found non-image file {non_images[0]} - This program doesn't allow it. Please use the '🚮 Clean folder' cell to remove non-image files.")
    elif img_count == 0:
      print(f"💥 Error: No images found in {images_folder}")
    else:
      print("\n💿 Analyzing dataset with FiftyOne...\n")
      dataset = fo.Dataset.from_dir(images_folder, dataset_type=fo.types.ImageDirectory)

      if len(dataset) == 0: # ADDED CHECK: Check if FiftyOne dataset is empty after loading
          print(f"💥 Error: FiftyOne could not load any valid images from '{images_folder}'.")
          print("Possible reasons: No supported image files, corrupted image files, or file access issues.")
          print("Please check your image directory and ensure it contains valid and supported image files (e.g., .png, .jpg, .jpeg, .gif).")
          enable_semantic_duplicate_finding = False # Stop further processing if no images loaded
      else: # Proceed only if dataset is not empty
          model = foz.load_zoo_model(model_name)
          embeddings = dataset.compute_embeddings(model, batch_size=batch_size)

          batch_embeddings = np.array_split(embeddings, batch_size)
          similarity_matrices = []
          max_size_x = max(array.shape[0] for array in batch_embeddings)
          max_size_y = max(array.shape[1] for array in batch_embeddings)

          for i, batch_embedding in enumerate(batch_embeddings):
            if batch_embedding.shape[0] > 0: # ADDED CHECK: Skip empty batches
              similarity = cosine_similarity(batch_embedding)
              #Pad 0 for np.concatenate
              padded_array = np.zeros((max_size_x, max_size_y))
              padded_array[0:similarity.shape[0], 0:similarity.shape[1]] = similarity
              similarity_matrices.append(padded_array)
            else:
              print(f"⚠️ Warning: Skipping empty batch {i} during similarity calculation.") # Optional warning

          if similarity_matrices: # Proceed only if similarity_matrices is not empty
            similarity_matrix = np.concatenate(similarity_matrices, axis=0)
            similarity_matrix = similarity_matrix[0:embeddings.shape[0], 0:embeddings.shape[0]]

            similarity_matrix = cosine_similarity(embeddings)
            similarity_matrix -= np.identity(len(similarity_matrix))

            dataset.match(F("max_similarity") > similarity_threshold)
            dataset.tags = ["delete", "has_duplicates"]

            id_map = [s.id for s in dataset.select_fields(["id"])]
            samples_to_remove = set()
            samples_to_keep = set()

            for idx, sample in enumerate(dataset):
              if sample.id not in samples_to_remove:
                # Keep the first instance of two duplicates
                samples_to_keep.add(sample.id)

                dup_idxs = np.where(similarity_matrix[idx] > similarity_threshold)[0]
                for dup in dup_idxs:
                    # We kept the first instance so remove all other duplicates
                    samples_to_remove.add(id_map[dup])

                if len(dup_idxs) > 0:
                    sample.tags.append("has_duplicates")
                    sample.save()
              else:
                sample.tags.append("delete")
                sample.save()

            clear_output()

            # sidebar_groups = fo.DatasetAppConfig.default_groups(dataset) # Corrected: default_groups instead of default_sidebar_groups for FiftyOne > 0.17 # REMOVED due to AttributeError
            # for group in sidebar_groups[1:]:
            #   group.expanded = False
            # dataset.app_config.sidebar_groups = sidebar_groups # REMOVED due to AttributeError
            dataset.save()
            session = fo.launch_app(dataset)

            print("❗ Wait a minute for the FiftyOne session to load. If it doesn't load after a few minutes:")
            print("- Try enabling cookies and removing tracking protection for the Google Colab website in your browser settings, as they may interfere with FiftyOne.")
            print("- If issues persist, check the FiftyOne documentation for Colab troubleshooting.")
            print("❗ Once loaded, you'll see a grid of your images in the FiftyOne App.")
            print("❗ To visualize images marked for deletion, enable \"sample tags\" in the left sidebar and select 'delete'.")
            print("❗ You can manually mark more images for deletion by selecting them and pressing the tag icon at the top, then adding the 'delete' tag.")
            input("⭕ When you're done reviewing and tagging in the FiftyOne App, press Enter here to save changes and remove duplicates: ")

            print("💾 Saving changes and removing marked duplicates...")

            marked = [s for s in dataset if "delete" in s.tags]
            dataset.delete_samples(marked)
            previous_folder = images_folder[:images_folder.rfind("/")]
            dataset.export(export_dir=os.path.join(images_folder, project_subfolder), dataset_type=fo.types.ImageDirectory)

            temp_suffix = "_temp"
            !mv {images_folder} {images_folder}{temp_suffix}
            !mv {images_folder}{temp_suffix}/{project_subfolder} {images_folder}
            !rm -r {images_folder}{temp_suffix}

            session.refresh()
            fo.close_app()
            clear_output()

            print(f"\n✅ Removed {len(marked)} images marked as duplicates from dataset. You now have {len(os.listdir(images_folder))} images.")
            print("✅ FiftyOne session closed.")
          else:
            print("⚠️ Warning: No similarity matrices calculated. Possible issue with embeddings or batch processing.")


else:
    print("Semantic duplicate image finding is disabled. Check the 'enable_semantic_duplicate_finding' checkbox to run.")

In [ ]:
#@title 🔄 Clean Dataset (Run after scraping)

import os
from PIL import Image # Import Pillow for image operations
# No need to import drive here, already done in the init cell

# Define directory using the initialized full_directory
# OUTPUT_BASE_DIR = "/content/drive/MyDrive/" # Removed, already defined and used in init cell
# subdirectory = "images" # Removed, already defined and used in init cell
# full_directory = os.path.join(OUTPUT_BASE_DIR, subdirectory) # Removed, already defined in init cell

# Feature Checkboxes
remove_duplicate_tags = True #@param {type:"boolean"}
remove_video_files = True #@param {type:"boolean"}
clean_tag_structure = True #@param {type:"boolean"}
remove_specific_tags = False #@param {type:"boolean"}
tags_to_remove_str = "deleteme" #@param {type:"string"}
remove_aspect_ratio_outliers = True #@param {type:"boolean"}
max_aspect_ratio_threshold = 0.25 #@param {type:"slider", min:0.01, max:0.5, step:0.01}
downscale_images = True #@param {type:"boolean"}
downscale_size = 1024 #@param {type:"slider", min:512, max:2048, step:64}

tags_to_remove = [tag.strip() for tag in tags_to_remove_str.split(',') if tag.strip()]

TEXT_EMOJI_PATTERNS = ["@_@", "^_^", "o_o", "x_x", ">_<", "u_u", "-_-"] # Add more if needed

# --- ANSI Color Codes ---
COLOR_RESET = "\033[0m"
COLOR_INFO = "\033[36m"     # Cyan for info messages
COLOR_WARNING = "\033[33m"  # Yellow for warnings
COLOR_SUCCESS = "\033[32m"  # Green for success

# --- Emojis ---
EMOJI_CLEAN = "🔄"
EMOJI_VIDEO = "🎬"
EMOJI_TAGS = "🏷️"
EMOJI_DUPLICATE = "👯"
EMOJI_ASPECT_RATIO = "📏"
EMOJI_DOWNSCALE = "⬇️"
EMOJI_ORPHANED_TAGS = "💔"
EMOJI_SUMMARY = "📊"
EMOJI_CHECKMARK = "✅"

def clean_tag_text(tag_string):
    """Cleans up the tag string, but preserves underscores in text emojis."""
    tag_string = tag_string.replace('\\(', '(').replace('\\)', ')')

    is_emoji = False
    for emoji_pattern in TEXT_EMOJI_PATTERNS:
        if tag_string == emoji_pattern:
            is_emoji = True
            break

    if not is_emoji:
        tag_string = tag_string.replace('_', ' ')

    tag_string = tag_string.replace('(', '\(').replace(')', '\)')
    return tag_string

def remove_duplicate_tags_from_file(tag_path):
    """Removes duplicate tags from a tag file."""
    try:
        with open(tag_path, 'r') as f:
            tags = f.read().strip()
        if not tags: # if file is empty
            return False

        tag_list = [tag.strip() for tag in tags.split(',')]
        unique_tags = sorted(list(set(tag_list))) # Remove duplicates and sort
        updated_tag_string = ', '.join(unique_tags)

        with open(tag_path, 'w') as f:
            f.write(updated_tag_string)
        return True # Modification happened
    except Exception as e:
        print(f"Error processing tag file {tag_path} for duplicate removal: {e}")
        return False

def clean_tag_structure_in_file(tag_path):
    """Cleans the tag structure within a tag file."""
    try:
        with open(tag_path, 'r') as f:
            tags = f.read().strip()
        if not tags: # if file is empty
            return False

        cleaned_tags = [clean_tag_text(tag.strip()) for tag in tags.split(',')]
        updated_tag_string = ', '.join(cleaned_tags)

        with open(tag_path, 'w') as f:
            f.write(updated_tag_string)
        return True # Modification happened
    except Exception as e:
        print(f"Error processing tag file {tag_path} for tag structure cleaning: {e}")
        return False

def remove_specific_tags_from_file(tag_path, tags_to_remove):
    """Removes specific tags from a tag file."""
    try:
        with open(tag_path, 'r') as f:
            tags_string = f.read().strip()
        if not tags_string: # if file is empty
            return False

        tag_list = [tag.strip() for tag in tags_string.split(',')]
        filtered_tags = [tag for tag in tag_list if tag not in tags_to_remove]
        updated_tag_string = ', '.join(filtered_tags)

        with open(tag_path, 'w') as f:
            f.write(updated_tag_string)
        return True # Modification happened
    except Exception as e:
        print(f"Error processing tag file {tag_path} for specific tag removal: {e}")
        return False

def remove_aspect_ratio_outliers_from_file(image_path, threshold):
    """Removes images with extreme aspect ratios."""
    try:
        img = Image.open(image_path)
        width, height = img.size
        aspect_ratio_wh = width / height
        aspect_ratio_hw = height / width

        if aspect_ratio_wh > (1 / threshold) or aspect_ratio_hw > (1 / threshold): # e.g., if threshold is 0.1, then > 10
            return True # Image is an aspect ratio outlier
        return False # Image is within aspect ratio threshold
    except Exception as e:
        print(f"Error processing image {image_path} for aspect ratio check: {e}")
        return False # Assume not an outlier in case of error

def downscale_image(image_path, target_size):
    """Downscales an image so its smallest dimension is target_size, if necessary."""
    try:
        img = Image.open(image_path)
        width, height = img.size

        if min(width, height) > target_size:
            if width < height:
                new_width = target_size
                new_height = int(height * (new_width / width))
            else:
                new_height = target_size
                new_width = int(width * (new_height / height))

            resized_img = img.resize((new_width, new_height), Image.LANCZOS) # Use LANCZOS for high-quality downscaling
            resized_img.save(image_path) # Overwrite the original image
            return True # Image was downscaled
        return False # Image was not downscaled
    except Exception as e:
        print(f"Error downscaling image {image_path}: {e}")
        return False # Indicate downscaling failed


def delete_orphan_tag_files(directory=full_directory, remove_video=False, clean_tags=False, remove_dupes=False, remove_spec_tags=False, tags_to_remove=[], remove_aspect_ratio=False, aspect_ratio_threshold=0.1, downscale_images=False, downscale_size=1024): # Use full_directory directly
    """
    Deletes .txt files in the given directory that do not have a corresponding image file
    and optionally removes video files, cleans tag structure and removes duplicate tags.
    """
    if not os.path.exists(directory):
        print(f"{COLOR_WARNING}Directory not found: {directory}{COLOR_RESET}")
        return

    print(f"\n--- {EMOJI_CLEAN} Cleaning Dataset in Progress {EMOJI_CLEAN} ---")
    print(f"Processing files in: {directory}")
    tag_files_deleted = 0
    video_files_deleted = 0
    tags_cleaned_count = 0
    dupes_removed_count = 0
    spec_tags_removed_count = 0
    aspect_ratio_outliers_removed_count = 0
    downscaled_images_count = 0

    video_extensions = ['.mp4', '.gif', '.webm', '.mov', '.avi'] # Add more video extensions if needed
    image_extensions = ['.jpg', '.jpeg', '.png', '.webp'] # Image extensions for aspect ratio check

    if remove_video:
        print(f"{EMOJI_VIDEO} Removing video files...")
        for filename in os.listdir(directory):
            for ext in video_extensions:
                if filename.lower().endswith(ext):
                    video_path = os.path.join(directory, filename)
                    tag_filename = os.path.splitext(filename)[0] + ".txt"
                    tag_path = os.path.join(directory, tag_path) # Corrected typo tag_path instead of filename

                    try:
                        os.remove(video_path)
                        video_files_deleted += 1
                        if os.path.exists(tag_path):
                            os.remove(tag_path) # Delete tag file too if video is removed
                            tag_files_deleted += 1 # Count as orphaned tag deleted in video removal
                        print(f"Deleted video file: {filename} and associated tag file (if any).")
                    except OSError as e:
                        print(f"Error deleting {filename}: {e}")
                        continue # Continue to next file even if error

    if clean_tags:
        print(f"{EMOJI_TAGS} Cleaning tag structure in tag files...")
        for filename in os.listdir(directory):
            if filename.endswith(".txt"):
                tag_path = os.path.join(directory, filename)
                if clean_tag_structure_in_file(tag_path):
                    tags_cleaned_count += 1

    if remove_dupes:
        print(f"{EMOJI_DUPLICATE} Removing duplicate tags in tag files...")
        for filename in os.listdir(directory):
            if filename.endswith(".txt"):
                tag_path = os.path.join(directory, filename)
                if remove_duplicate_tags_from_file(tag_path):
                    dupes_removed_count += 1

    if remove_spec_tags:
        print(f"{EMOJI_TAGS} Removing specific tags: {tags_to_remove} from tag files...")
        for filename in os.listdir(directory):
            if filename.endswith(".txt"):
                tag_path = os.path.join(directory, filename)
                if remove_specific_tags_from_file(tag_path, tags_to_remove):
                    spec_tags_removed_count += 1

    if remove_aspect_ratio:
        print(f"{EMOJI_ASPECT_RATIO} Removing images with extreme aspect ratios (threshold: 1:{1/aspect_ratio_threshold:.0f})...")
        for filename in os.listdir(directory):
            for ext in image_extensions:
                if filename.lower().endswith(ext):
                    image_path = os.path.join(directory, filename)
                    tag_filename = os.path.splitext(filename)[0] + ".txt"
                    tag_path = os.path.join(directory, tag_filename)

                    if remove_aspect_ratio_outliers_from_file(image_path, aspect_ratio_threshold):
                        try:
                            os.remove(image_path)
                            aspect_ratio_outliers_removed_count += 1
                            if os.path.exists(tag_path):
                                os.remove(tag_path) # Delete tag file too if image is removed
                                tag_files_deleted += 1 # Count as orphaned tag deleted in image removal
                            print(f"Deleted image with extreme aspect ratio: {filename} and associated tag file (if any).")
                        except OSError as e:
                            print(f"Error deleting {filename}: {e}")
                        break # Only process image files once

    if downscale_images:
        print(f"{EMOJI_DOWNSCALE} Downscaling images to max dimension: {downscale_size}...")
        for filename in os.listdir(directory):
            for ext in image_extensions:
                if filename.lower().endswith(ext):
                    image_path = os.path.join(directory, filename)
                    if downscale_image(image_path, downscale_size):
                        downscaled_images_count += 1
                        print(f"Downscaled image: {filename}")


    print("Checking for orphaned text files...")
    for filename in os.listdir(directory):
        if filename.endswith(".txt"):
            base_filename = os.path.splitext(filename)[0]
            tag_path = os.path.join(directory, filename)
            image_exists = False

            #print(f"\nChecking tag file: {filename}") # DEBUG PRINT
            #print(f"  Base filename: {base_filename}") # DEBUG PRINT
            #print(f"  Tag path: {tag_path}") # DEBUG PRINT

            # Check for common image file extensions (excluding GIFs for orphan check as GIFs can be videos)
            for ext in ['.jpg', '.jpeg', '.png', '.webp']: # Exclude .gif here
                image_filename = base_filename + ext
                image_path = os.path.join(directory, image_filename)
                #print(f"  Checking for image: {image_filename}") # DEBUG PRINT
                #print(f"  Image path: {image_path}") # DEBUG PRINT
                if os.path.exists(image_path):
                    image_exists = True
                    #print(f"  Image EXISTS: {image_path}") # DEBUG PRINT
                    break # No need to check other extensions if image is found
                #else:
                    #print(f"  Image DOES NOT EXIST: {image_path}") # DEBUG PRINT

            if not image_exists:
                #print(f"  Orphaned tag file FOUND: {filename}") # DEBUG PRINT
                try:
                    os.remove(tag_path)
                    print(f"  Deleted orphaned tag file: {filename}")
                    tag_files_deleted += 1
                except OSError as e:
                    print(f"Error deleting {filename}: {e}")

    print("\n--- Cleanup Summary ---")
    if video_files_deleted > 0:
        print(f"{EMOJI_VIDEO} Deleted video files: {video_files_deleted}")
    if tags_cleaned_count > 0:
        print(f"{EMOJI_TAGS} Cleaned tag structure in tag files: {tags_cleaned_count}")
    if dupes_removed_count > 0:
        print(f"{EMOJI_DUPLICATE} Removed duplicate tags from tag files: {dupes_removed_count}")
    if spec_tags_removed_count > 0:
        print(f"{EMOJI_TAGS} Removed specific tags from tag files: {spec_tags_removed_count}")
    if aspect_ratio_outliers_removed_count > 0:
        print(f"{EMOJI_ASPECT_RATIO} Removed images with extreme aspect ratios: {aspect_ratio_outliers_removed_count}")
    if downscaled_images_count > 0:
        print(f"{EMOJI_DOWNSCALE} Downscaled images to max dimension {downscale_size}px: {downscaled_images_count}")
    if tag_files_deleted > 0 and not remove_video and not remove_aspect_ratio: # Don't double count if tags were deleted during video or aspect ratio removal
        print(f"{EMOJI_ORPHANED_TAGS} Deleted orphaned tag files: {tag_files_deleted}")
    elif tag_files_deleted == 0 and not video_files_deleted and tags_cleaned_count == 0 and dupes_removed_count == 0 and spec_tags_removed_count == 0 and aspect_ratio_outliers_removed_count == 0 and downscaled_images_count == 0:
        print(f"{EMOJI_CHECKMARK} No orphaned tag files, video files, tag structure issues, duplicate tags, specific tags to remove, extreme aspect ratio images, or images to downscale found.")
    else:
        print(f"{EMOJI_ORPHANED_TAGS} No additional orphaned tag files found (after video/aspect ratio/downscaling removal).")

    print(f"{EMOJI_CHECKMARK} {COLOR_SUCCESS}Dataset cleaning process completed.{COLOR_RESET}")


# Run the function with checkbox parameters
delete_orphan_tag_files(directory=full_directory, remove_video=remove_video_files, clean_tags=clean_tag_structure, remove_dupes=remove_duplicate_tags, remove_spec_tags=remove_specific_tags, tags_to_remove=tags_to_remove, remove_aspect_ratio=remove_aspect_ratio_outliers, aspect_ratio_threshold=max_aspect_ratio_threshold, downscale_images=downscale_images, downscale_size=downscale_size) # Use full_directory

In [ ]:
#@title 🔁 Tag Replacer

import os

# No need to import drive or define directories again
# Define directory using the initialized full_directory
# full_directory is assumed to be defined in the "Initialization Cell"

# Colab Parameters
tags_to_replace_str = "1girl" #@param {type:"string"}
replacement_tag = "1woman" #@param {type:"string"}
case_insensitive = True #@param {type:"boolean"}

tags_to_replace = [tag.strip() for tag in tags_to_replace_str.split(',') if tag.strip()]

# --- ANSI Color Codes ---
COLOR_RESET = "\033[0m"
COLOR_WARNING = "\033[33m"  # Yellow for warnings
COLOR_SUCCESS = "\033[32m"  # Green for success

# --- Emojis ---
EMOJI_REPLACE = "🔁"
EMOJI_WARNING = "⚠️"
EMOJI_CHECKMARK = "✅"
EMOJI_FILES = "🗂️"
EMOJI_TAGS = "🏷️"

def replace_tags_in_directory(directory=full_directory, tags_to_replace=[], replacement_tag="", case_insensitive=True):
    """
    Replaces specified tags with a new tag in all .txt files in the given directory, with enhanced output.
    """
    if not os.path.exists(directory):
        print(f"{COLOR_WARNING}Directory not found: {directory}{COLOR_RESET}") # Color added
        return

    if not tags_to_replace or not replacement_tag:
        print(f"{EMOJI_WARNING} {COLOR_WARNING}Warning: No tags to replace or replacement tag specified. No changes will be made.{COLOR_RESET}") # Emoji and color added
        return

    print(f"\n--- {EMOJI_REPLACE} Tag Replacement in Progress {EMOJI_REPLACE} ---")
    print(f"{EMOJI_FOLDER} Processing directory: {directory}")
    print(f"{EMOJI_TAGS} Tags to replace: {tags_to_replace}")
    print(f"{EMOJI_TAGS} Replacement tag: '{replacement_tag}'")
    if case_insensitive:
        print("Performing case-insensitive replacement.")

    files_modified_count = 0
    tags_replaced_count = 0

    for filename in os.listdir(directory):
        if filename.endswith(".txt"):
            tag_path = os.path.join(directory, filename)
            try:
                with open(tag_path, 'r') as f:
                    tags_string = f.read().strip()
                if not tags_string:
                    continue # Skip empty tag files

                tag_list = [tag.strip() for tag in tags_string.split(',')]
                updated_tags = []
                file_modified = False
                for tag in tag_list:
                    found_match = False
                    for tag_to_replace in tags_to_replace:
                        if case_insensitive:
                            if tag.lower() == tag_to_replace.lower():
                                updated_tags.append(replacement_tag)
                                tags_replaced_count += 1
                                file_modified = True
                                found_match = True
                                break # Break inner loop once a match is found for case-insensitive
                        else:
                            if tag == tag_to_replace:
                                updated_tags.append(replacement_tag)
                                tags_replaced_count += 1
                                file_modified = True
                                found_match = True
                                break # Break inner loop once a match is found for case-sensitive
                    if not found_match:
                        updated_tags.append(tag) # Keep original tag if no replacement

                if file_modified:
                    updated_tag_string = ', '.join(updated_tags)
                    with open(tag_path, 'w') as f:
                        f.write(updated_tag_string)
                    files_modified_count += 1

            except Exception as e:
                print(f"Error processing tag file {filename}: {e}")

    print(f"\n--- {EMOJI_REPLACE} Tag Replacement Summary {EMOJI_REPLACE} ---")
    print(f"{EMOJI_FILES} Files modified: {files_modified_count}")
    print(f"{EMOJI_TAGS} Tags replaced: {tags_replaced_count}")
    print(f"{EMOJI_CHECKMARK} {COLOR_SUCCESS}Tag replacement process completed.{COLOR_RESET}")


# Run tag replacement with Colab parameters
replace_tags_in_directory(directory=full_directory, tags_to_replace=tags_to_replace, replacement_tag=replacement_tag, case_insensitive=case_insensitive)

In [ ]:
#@title 📈 Analyze Tags

import os
from collections import Counter
# No need to import drive or define directories again
import requests # Import requests for get_tag_data
import json # Import json for get_tag_data

# Define directory using the initialized full_directory
# full_directory is assumed to be defined in the "Initialization Cell"

# Analysis Parameters
display_type = "Top" #@param ["Top", "Bottom", "Alphabetical"]
num_tags_to_display = 100 #@param {type:"integer"}
threshold_type = "Minimum Frequency" #@param ["None", "Minimum Frequency", "Maximum Frequency", "Longer Than", "Shorter Than"]
threshold_value = 5 #@param {type:"integer"}
analyze_category = "All" #@param ["All", "General", "Artist", "Meta", "Character", "Copyright", "Unknown"]

# --- ANSI Color Codes ---
COLOR_RESET = "\033[0m"
COLOR_UNKNOWN = "\033[31m"    # Red - ADDED COLOR_UNKNOWN definition here
COLOR_CATEGORY = {
    "general": "\033[34m",    # Blue
    "artist": "\033[32m",     # Green
    "meta": "\033[33m",       # Yellow
    "character": "\033[35m",  # Magenta
    "copyright": "\033[36m",  # Cyan
    "unknown": COLOR_UNKNOWN,    # Red - Use COLOR_UNKNOWN here for "unknown" category too, for consistency
}

# --- Emojis ---
EMOJI_SUMMARY = "📊"
EMOJI_TOP_TAGS = "🏆"
EMOJI_BOTTOM_TAGS = "📉"
EMOJI_ALPHABETICAL_TAGS = "🔤" # Added emoji for alphabetical sort
EMOJI_CATEGORY_BREAKDOWN = "🗂️"

def unclean_tag_text(tag_string):
    """Reverts cleaned tag string back to original format for API lookup."""
    tag_string = tag_string.replace(' ', '_') # Replace spaces with underscores
    tag_string = tag_string.replace('\\(', '(').replace('\\)', ')') # Remove backslashes from parentheses
    return tag_string


def analyze_tags(directory=full_directory, display_type="Top", num_tags_to_display=10, threshold_type="None", threshold_value=0, analyze_category="All"): # Renamed params
    """
    Analyzes tag frequencies in .txt files, with frequency and length thresholds, and category filtering.
    """
    if not os.path.exists(directory):
        print(f"Directory not found: {directory}")
        return

    print(f"Analyzing tags in: {directory}")
    if analyze_category != "All":
        print(f"Filtering by category: {analyze_category}")

    tag_counts = Counter()
    tag_files_analyzed = 0

    all_tags_with_categories = {} # Store tags with their categories

    for filename in os.listdir(directory):
        if filename.endswith(".txt"):
            tag_files_analyzed += 1
            tag_path = os.path.join(directory, filename)
            try:
                with open(tag_path, 'r') as f:
                    tags_string = f.read().strip()
                    if tags_string:
                        tags = [tag.strip() for tag in tags_string.split(',')]
                        tag_counts.update(tags)
                        # Unclean tags for API lookup to get categories
                        uncleaned_tags_for_api = [unclean_tag_text(tag) for tag in tags]
                        tag_categories = get_tag_categories(uncleaned_tags_for_api) # Get categories for UNCLEANED tags
                        # Store category for each *cleaned* tag name
                        for i, tag_name in enumerate(tags):
                            original_tag_name_api = unclean_tag_text(tag_name) # Unclean tag to match API response
                            all_tags_with_categories[tag_name] = tag_categories.get(original_tag_name_api, 'unknown') # Get category using unclean name

            except Exception as e:
                print(f"Error reading tag file {filename}: {e}")

    total_unique_tags = len(tag_counts)
    print(f"\n--- {EMOJI_SUMMARY} Tag Analysis Summary {EMOJI_SUMMARY} ---")
    print(f"Tag files analyzed: {tag_files_analyzed}")
    print(f"Total unique tags found: {total_unique_tags}")

    filtered_tags = tag_counts.copy()

    if threshold_type == "Minimum Frequency": # Renamed option
        filtered_tags = Counter({tag: count for tag, count in tag_counts.items() if count >= threshold_value})
        print(f"Applying Minimum frequency threshold: {threshold_value}")
    elif threshold_type == "Maximum Frequency": # Renamed option
        filtered_tags = Counter({tag: count for tag, count in tag_counts.items() if count <= threshold_value})
        print(f"Applying Maximum frequency threshold: {threshold_value}")
    elif threshold_type == "Longer Than": # New length threshold
        filtered_tags = Counter({tag: count for tag, count in tag_counts.items() if len(tag) > threshold_value})
        print(f"Filtering tags longer than {threshold_value} characters.")
    elif threshold_type == "Shorter Than": # New length threshold
        filtered_tags = Counter({tag: count for tag, count in tag_counts.items() if len(tag) < threshold_value})
        print(f"Filtering tags shorter than {threshold_value} characters.")


    if analyze_category != "All":
        category_filtered_tags = Counter()
        for tag, count in filtered_tags.items():
            if all_tags_with_categories.get(tag) == analyze_category.lower():
                category_filtered_tags[tag] = count
        filtered_tags = category_filtered_tags


    num_filtered_unique_tags = len(filtered_tags)
    print(f"Unique tags after filtering: {num_filtered_unique_tags}")

    if not filtered_tags:
        print("\nNo tags to display after applying filters.")
        return

    if display_type == "Top":
        most_common_tags = filtered_tags.most_common(num_tags_to_display)
        print(f"\n--- {EMOJI_TOP_TAGS} Top {num_tags_to_display} Most Frequent Tags ({analyze_category} Category) {EMOJI_TOP_TAGS} ---") # Emoji added
    elif display_type == "Bottom":
        least_common_tags_list = filtered_tags.most_common()[:-num_tags_to_display-1:-1] # Get least common, reverse order
        least_common_tags = reversed(least_common_tags_list)
        print(f"\n--- {EMOJI_BOTTOM_TAGS} Bottom {num_tags_to_display} Least Frequent Tags ({analyze_category} Category) {EMOJI_BOTTOM_TAGS} ---") # Emoji added
    elif display_type == "Alphabetical":
        alphabetical_tags_list = sorted(filtered_tags.keys())
        display_tag_list = [(tag, filtered_tags[tag]) for tag in alphabetical_tags_list[:num_tags_to_display]] # Get counts for sorted tags, limit to num_tags_to_display
        print(f"\n--- {EMOJI_ALPHABETICAL_TAGS} Top {num_tags_to_display} Tags (Alphabetical Order, {analyze_category} Category) {EMOJI_ALPHABETICAL_TAGS} ---") # Emoji added
    else:
        print("Invalid display type selected.")
        return

    if display_type == "Top":
      display_tag_list = most_common_tags
    elif display_type == "Bottom":
      display_tag_list = least_common_tags
    elif display_type == "Alphabetical":
      pass

    for tag, count in display_tag_list:
        category = all_tags_with_categories.get(tag, 'unknown')
        color_code = COLOR_CATEGORY.get(category, COLOR_UNKNOWN)
        print(f"- {tag}: {count} ({color_code}{category.capitalize()}{COLOR_RESET})")

    print(f"\n--- {EMOJI_CATEGORY_BREAKDOWN} Tag Category Breakdown {EMOJI_CATEGORY_BREAKDOWN} ---")
    category_counts = Counter()
    for tag in filtered_tags.keys():
        category_counts[all_tags_with_categories.get(tag, 'unknown')] += filtered_tags[tag]

    for category, count in category_counts.most_common():
        color_code = COLOR_CATEGORY.get(category, COLOR_UNKNOWN)
        print(f"- {color_code}{category.capitalize()}{COLOR_RESET}: {count} tags")


def get_tag_categories(tags_list):
    """
    Retrieves tag categories for a list of tags using the Gelbooru API.
    Returns a dictionary mapping tag names to categories.
    """
    tags_string = ' '.join(tags_list)
    tag_data_list = get_tag_data(tags_string)

    tag_categories = {}
    for tag_info in tag_data_list:
        tag_categories[tag_info['name']] = tag_info['category']
    return tag_categories


def get_tag_data(tags_string):
    """
    Retrieves tag data with categories using the Gelbooru API.
    (Copied from scraper cell for tag category analysis)
    """

    api_url = "https://gelbooru.com/index.php"
    params = {
        "page": "dapi",
        "s": "tag",
        "q": "index",
        "json": 1,
        "names": tags_string
    }

    try:
      response = requests.get(api_url, params=params)
      response.raise_for_status()
      tag_data = response.json()

      # Check if the response is a dictionary and convert it to a list
      if isinstance(tag_data, dict) and 'tag' in tag_data:
        tag_data = tag_data['tag']
      elif not isinstance(tag_data, list):
        print(f"Unexpected API response format: {tag_data}")
        return []

      # Map Gelbooru's numerical tag types to category names
      tag_type_mapping = {
          0: "general",
          1: "artist",
          3: "copyright",
          4: "character",
          5: "meta",
        }

      # Extract tag names and categories
      tag_info = []
      for tag in tag_data:
          tag_info.append({
            "name": tag["name"],
            "category": tag_type_mapping.get(tag["type"], "unknown")
          })

      return tag_info
    except requests.exceptions.RequestException as e:
      print(f"Error fetching tag data: {e}")
      return []

# Run tag analysis with Colab parameters
analyze_tags(directory=full_directory, display_type=display_type, num_tags_to_display=num_tags_to_display, threshold_type=threshold_type, threshold_value=threshold_value, analyze_category=analyze_category)

In [ ]:
#@title 🔍 Search for Tags

import os

# No need to import drive or define directories again
# Define directory using the initialized full_directory
# full_directory is assumed to be defined in the "Initialization Cell"

# Colab Parameters
search_tags_str = "*girl" #@param {type:"string"}
case_insensitive_search = True #@param {type:"boolean"}
list_files_with_tags = True #@param {type:"boolean"}

search_tags = [tag.strip() for tag in search_tags_str.split(',') if tag.strip()]

# --- ANSI Color Codes ---
COLOR_RESET = "\033[0m"
COLOR_HIGHLIGHT = "\033[33m" # Yellow highlight for search terms

# --- Emojis ---
EMOJI_SEARCH = "🔍"
EMOJI_TAG = "🏷️"
EMOJI_FILE = "📄"

def search_tags_in_directory(directory=full_directory, search_tags=[], case_insensitive_search=True, list_files_with_tags=True): # list_files_with_tags defaults to True
    """
    Searches for specified tags (with wildcard support) in .txt files, counts occurrences,
    displays matched tags, and lists files with the tags and their contained tags.
    """
    if not os.path.exists(directory):
        print(f"Directory not found: {directory}")
        return

    if not search_tags:
        print("⚠️ Warning: No tags specified to search for.")
        return

    print(f"\n--- {EMOJI_SEARCH} Tag Search in Progress (Wildcard Enabled) {EMOJI_SEARCH} ---")
    print(f"Searching for tags: {search_tags}")
    print(f"Directory: {directory}")
    if case_insensitive_search:
        print("Performing case-insensitive search.")

    tag_occurrences = {tag: 0 for tag in search_tags} # Initialize counts for each search tag
    files_with_tags = {tag: {} for tag in search_tags} # Modified: Store files as dicts, tag: {filename: [matched_tags]}
    matched_tags_list = {tag: [] for tag in search_tags} # Store matched tags for wildcard searches

    files_analyzed_count = 0

    for filename in os.listdir(directory):
        if filename.endswith(".txt"):
            files_analyzed_count += 1
            tag_path = os.path.join(directory, filename)
            try:
                with open(tag_path, 'r') as f:
                    tags_string = f.read().strip()
                if not tags_string:
                    continue # Skip empty tag files

                tag_list = [tag.strip() for tag in tags_string.split(',')]

                for search_tag in search_tags:
                    file_matched_tags = [] # NEW: List to store tags matched in current file
                    found_in_file = False
                    for tag in tag_list:
                        if "*" in search_tag: # Wildcard search logic
                            prefix = search_tag.split("*")[0]
                            if case_insensitive_search:
                                if tag.lower().startswith(prefix.lower()):
                                    tag_occurrences[search_tag] += 1
                                    matched_tags_list[search_tag].append(tag)
                                    file_matched_tags.append(tag) # NEW: Add matched tag to file_matched_tags
                                    found_in_file = True
                            else:
                                if tag.startswith(prefix):
                                    tag_occurrences[search_tag] += 1
                                    matched_tags_list[search_tag].append(tag)
                                    file_matched_tags.append(tag) # NEW: Add matched tag to file_matched_tags
                                    found_in_file = True
                        else: # Exact match search logic (no wildcard)
                            if case_insensitive_search:
                                if tag.lower() == search_tag.lower():
                                    tag_occurrences[search_tag] += 1
                                    found_in_file = True
                                    break # No need to count multiple times in same file (case-insensitive)
                            else:
                                if tag == search_tag:
                                    tag_occurrences[search_tag] += 1
                                    found_in_file = True
                                    break # No need to count multiple times in same file (case-sensitive)
                    if list_files_with_tags and found_in_file:
                        files_with_tags[search_tag][filename] = file_matched_tags # MODIFIED: Store filename and matched_tags


            except Exception as e:
                print(f"Error processing tag file {filename}: {e}")

    print(f"\n--- {EMOJI_SEARCH} Tag Search Summary {EMOJI_SEARCH} ---")
    print(f"Files analyzed: {files_analyzed_count}")
    for search_tag, count in tag_occurrences.items():
        print(f"\n{EMOJI_TAG} Tag: {COLOR_HIGHLIGHT}{search_tag}{COLOR_RESET}") # Emoji and color added
        print(f"  Occurrences: {count}")
        if "*" in search_tag: # NEW: Display matched tags for wildcard searches
            matched_tags = matched_tags_list[search_tag]
            if matched_tags:
                print(f"  Matched tags: {', '.join(matched_tags)}") # NEW: List matched tags
        if list_files_with_tags:
            files_and_tags = files_with_tags[search_tag] # MODIFIED: Get files_with_tags dict
            if files_and_tags:
                print(f"  Found in files ({len(files_and_tags)}):")
                for file, matched_tags in files_and_tags.items(): # MODIFIED: Iterate through files_and_tags.items()
                    print(f"  - {EMOJI_FILE} {file}: Tags: {', '.join(matched_tags)}") # MODIFIED: Display filename and matched tags
            else:
                print("  Found in no files.") # Should not happen if count > 0, but for completeness

    print(f"\n{EMOJI_CHECKMARK} Tag search process completed.")


# Run tag search with Colab parameters
search_tags_in_directory(directory=full_directory, search_tags=search_tags, case_insensitive_search=case_insensitive_search, list_files_with_tags=list_files_with_tags)

In [ ]:
#@title 🔢 Count Dataset

import os

# No need to import drive or define directories again
# Define directory using the initialized full_directory
# full_directory is assumed to be defined in the "Initialization Cell"

# --- Emojis ---
EMOJI_SUMMARY = "📊"
EMOJI_FILES = "🗂️"
EMOJI_IMAGES = "🖼️"
EMOJI_TAGS = "🏷️"
EMOJI_SIZE = "💾"

def count_dataset(directory=full_directory): # Use full_directory directly
    """
    Counts and summarizes the files in the specified directory,
    including image files, text files, and the total size of image files, with enhanced output.
    """
    if not os.path.exists(directory):
        print(f"Directory not found: {directory}")
        return

    image_extensions = ['.jpg', '.jpeg', '.png', '.gif', '.webp']
    image_count = 0
    text_count = 0
    total_image_size = 0

    for filename in os.listdir(directory):
        file_path = os.path.join(directory, filename)
        if os.path.isfile(file_path):
            is_image = False
            for ext in image_extensions:
                if filename.lower().endswith(ext):
                    is_image = True
                    break
            if is_image:
                image_count += 1
                total_image_size += os.path.getsize(file_path)
            elif filename.endswith(".txt"):
                text_count += 1

    print(f"\n--- {EMOJI_SUMMARY} Dataset Summary {EMOJI_SUMMARY} ---")
    print(f"{EMOJI_FILES} Directory: {directory}")
    print(f"{EMOJI_FILES} Total Files: {image_count + text_count}")
    print(f"{EMOJI_IMAGES} Image Files: {image_count}")
    print(f"{EMOJI_TAGS} Tag Files (text): {text_count}")

    # Convert bytes to human-readable format (e.g., KB, MB, GB)
    size_in_gb = total_image_size / (1024**3)
    size_in_mb = total_image_size / (1024**2)
    size_in_kb = total_image_size / 1024

    if size_in_gb >= 1:
        print(f"{EMOJI_SIZE} Total Image Size: {size_in_gb:.2f} GB")
    elif size_in_mb >= 1:
        print(f"{EMOJI_SIZE} Total Image Size: {size_in_mb:.2f} MB")
    elif size_in_kb >= 1:
        print(f"{EMOJI_SIZE} Total Image Size: {size_in_kb:.2f} KB")
    else:
        print(f"{EMOJI_SIZE} Total Image Size: {total_image_size} bytes")

# Run the dataset counting function
count_dataset(directory=full_directory) # Use full_directory

In [ ]:
#@title 🚚 Move or Copy Dataset Files

import os
import shutil # Import shutil for copy operation

# No need to import drive or define directories again
# Define directory using the initialized full_directory
# full_directory is assumed to be defined in the "Initialization Cell"

# Colab Parameters
destination_path = "/content/drive/MyDrive/Path/To/Somewhere" #@param {type:"string"}
copy_files = False #@param {type:"boolean"}

def move_copy_dataset(source_dir=full_directory, dest_dir="", copy_files=False):
    """
    Moves or copies image and tag files from the source directory to the destination directory.

    Args:
        source_dir (str): Path to the source directory (default: full_directory).
        dest_dir (str): Path to the destination directory in Google Drive.
        copy_files (bool): If True, files are copied; if False, files are moved.
    """
    if not os.path.exists(source_dir):
        print(f"Error: Source directory not found: {source_dir}")
        return
    if not dest_dir:
        print("Error: Destination path is empty. Please provide a valid Google Drive path.")
        return

    os.makedirs(dest_dir, exist_ok=True)

    operation_type = "Copying" if copy_files else "Moving"
    print(f"\n--- {operation_type} Dataset Files ---")
    print(f"Source directory: {source_dir}")
    print(f"Destination directory: {dest_dir}")

    files_processed_count = 0
    errors_count = 0

    for filename in os.listdir(source_dir):
        source_file_path = os.path.join(source_dir, filename)
        dest_file_path = os.path.join(dest_dir, filename)

        if os.path.isfile(source_file_path):
            try:
                if copy_files:
                    shutil.copy2(source_file_path, dest_file_path)
                else:
                    shutil.move(source_file_path, dest_file_path)
                files_processed_count += 1
            except Exception as e:
                print(f"Error processing file {filename}: {e}")
                errors_count += 1

    print("\n--- Move/Copy Summary ---")
    print(f"Operation type: {operation_type}")
    print(f"Files processed: {files_processed_count}")
    if errors_count > 0:
        print(f"Errors encountered: {errors_count}")
    print(f"✅ Dataset files {operation_type.lower()} completed.")


# Run move/copy operation based on Colab parameters
move_copy_dataset(source_dir=full_directory, dest_dir=destination_path, copy_files=copy_files)

In [ ]:
#@title 🚮 Clean Folder

import os
# No need to import drive or define directories again

# Feature Checkboxes for folder cleaning
remove_non_image_files = True #@param {type:"boolean"}
delete_all_files = True #@param {type:"boolean"}

# --- Emojis ---
EMOJI_FOLDER = "🗂️"
EMOJI_DELETE = "🗑️"
EMOJI_NON_IMAGE = "📄"
EMOJI_ALL_FILES = "💥"
EMOJI_WARNING = "⚠️"
EMOJI_CHECKMARK = "✅"

def clean_folder(directory=full_directory, remove_non_images=False, delete_all=False):
    """
    Cleans the specified directory by removing non-image files or all files, with enhanced output.
    """
    if not os.path.exists(directory):
        print(f"{EMOJI_WARNING} Directory not found: {directory}")
        return

    if delete_all:
        print(f"{EMOJI_WARNING} Deleting ALL files in: {directory} - USE WITH CAUTION! {EMOJI_ALL_FILES}")
        file_count = 0
        for filename in os.listdir(directory):
            file_path = os.path.join(directory, filename)
            try:
                if os.path.isfile(file_path):
                    os.remove(file_path)
                    file_count += 1
            except OSError as e:
                print(f"Error deleting {filename}: {e}")
        print(f"{EMOJI_CHECKMARK} Deleted {file_count} files in total (all files). {EMOJI_DELETE}")

    elif remove_non_images:
        print(f"{EMOJI_DELETE} Removing non-image files from: {directory} {EMOJI_NON_IMAGE}")
        image_extensions = ['.jpg', '.jpeg', '.png', '.gif', '.webp']
        non_image_files_deleted = 0

        for filename in os.listdir(directory):
            is_image = False
            for ext in image_extensions:
                if filename.lower().endswith(ext):
                    is_image = True
                    break
            if not is_image and os.path.isfile(os.path.join(directory, filename)): # Check if it's a file and not an image
                try:
                    os.remove(os.path.join(directory, filename))
                    non_image_files_deleted += 1
                    print(f"Deleted non-image file: {filename}")
                except OSError as e:
                    print(f"Error deleting {filename}: {e}")
        print(f"{EMOJI_CHECKMARK} Deleted {non_image_files_deleted} non-image files. {EMOJI_NON_IMAGE}")

    else:
        print(f"{EMOJI_FOLDER} No cleaning options selected. Check the checkboxes to perform cleaning actions.")

# Run the folder cleaning function based on checkbox selections
clean_folder(directory=full_directory, remove_non_images=remove_non_image_files, delete_all=delete_all_files) # Use full_directory